In [13]:
from theano import tensor as T
import lasagne_dev as nn
import numpy as np
import pickle
from input_process import read_images
import random
import math
import cmath
from g_gradient import *
import timeit
import unet
from collections import OrderedDict

In [19]:
start = timeit.default_timer()
epoch = 1
shape = [5,1,116,116]

input_var = T.tensor4('input_var')  
label_var = T.tensor4('label_var')

[network, loss, test_loss, test_acc, output_det] = unet.network(input_var, label_var, shape)


params = nn.layers.get_all_params(network)
lr = theano.shared(nn.utils.floatX(1e-4)) # learning rate
updates = nn.updates.adam(loss,params, learning_rate=lr) # adam most widely used update scheme

gs = nn.layers.get_all_gs(network)

gs_updates = g_updates(loss, params, gs)
updates = OrderedDict()
updates_old = nn.updates.adam(loss, params, learning_rate=lr)

stop = timeit.default_timer()
print(stop-start)

0.27063217500108294


In [37]:
def gabor_filter_tensor_dev(x,y,params):
    f,gamma,sigma,theta,psi = params
    xt = x*theano.tensor.cos(theta) + y*theano.tensor.sin(theta)
    yt = -x*theano.tensor.sin(theta) + y*theano.tensor.cos(theta)
    z1 = -(xt**2 + (gamma*yt)**2)/(2*sigma**2)
    z2 = 2*math.pi*f*xt+psi
    #z2 = 1j*2*math.pi*f*xt+psi
    value = f**2/(math.pi*gamma)*theano.tensor.exp(z1)*theano.tensor.cos(z2)
    return value

def gabor_weight_update_dev(shape, gs):
    [num_filters,num_channels,size,size] = shape
    #Ws = np.array([], dtype=np.float32).reshape(1,-1)

    gfilter = []

    for filter_index in range (0,num_filters):
        for channel_index in range (0,num_channels):
            
            params = gs[filter_index, channel_index]
            g_params = [params[0],params[1],params[2],params[3],params[4]]

            
            bond = math.floor(size/2)
            x_range = np.linspace(-bond, bond, size)
            y_range = np.linspace(-bond, bond, size)

            [x_range,y_range] = list(map(lambda x:x.reshape(1,-1),np.meshgrid(x_range,y_range)))
            
            for (x,y) in zip(np.ndarray.tolist(x_range)[0], np.ndarray.tolist(y_range)[0]):
                value = gabor_filter_tensor_dev(x,y,g_params)
                #gfilter.append(value)
            
            
    #Ws = theano.tensor.stack(gfilter)
    #Ws = Ws.reshape(shape)
            #W = np.array(gfilter, dtype=np.float32)
            #W = np.array(gfilter)
            #W = W.reshape(1,-1)
            #Ws = np.concatenate((Ws,W),axis=1)
    return 0

In [74]:
gg = [1,2,3,4,5]
start = timeit.default_timer()
gabor_filter(1,1,gg)
stop = timeit.default_timer()
print(stop-start)


gg_shared = theano.shared(np.array(gg))

f,gamma,sigma,theta,psi = [gg_shared[0],gg_shared[1],gg_shared[2],gg_shared[3],gg_shared[4]]
x = 1
y = 1
    
start = timeit.default_timer()
xt = x*theano.tensor.cos(theta) + y*theano.tensor.sin(theta)
stop = timeit.default_timer()
print(stop-start)

start = timeit.default_timer()
yt = -x*theano.tensor.sin(theta) + y*theano.tensor.cos(theta)
stop = timeit.default_timer()
print(stop-start)

start = timeit.default_timer()
z1 = -(xt**2 + (gamma*yt)**2)/(2*sigma**2)
stop = timeit.default_timer()
print(stop-start)

start = timeit.default_timer()
z2 = 2*math.pi*f*xt+psi
stop = timeit.default_timer()
print(stop-start)


7.422899943776429e-05
0.002882180007873103
0.0034633189934538677
0.005202672007726505
0.0014443079999182373


In [41]:
gg = [1,2,3,4,5]
gg_shared = theano.shared(np.array(gg))
gg_shared[0].eval()

array(1)

In [33]:
start = timeit.default_timer()
## generate updates for params

for i in range (0,len(gs)):

    gs_new = gs_updates[gs[i]]
    ws = params[i*2]
    [num_filters, num_channels, filter_size, filter_size] = ws.get_value().shape
    W = gabor_weight_update_dev([num_filters, num_channels, filter_size, filter_size], gs_new)
    #updates[ws] = theano.shared(W)
    updates[ws] = W
    updates[params[i*2+1]] = updates_old[params[i*2+1]]

stop = timeit.default_timer()
print(stop-start)

27.818595878998167


In [10]:
from input_process import read_images
from train import run_params
import pickle
import numpy as np
import random

size = (116, 116) # resize images

pkl_file = open('../fnames_collection_2', 'r')
fnames = pkl_file.readline()
fnames = fnames.split(' ')
for i in range (0,len(fnames)):
	fnames[i] = str(fnames[i])
	fnames[i] = 'image' + fnames[i] + '.png'

'''
index = fnames[1:4000]
random.shuffle(index)
train_index = index[0:4000]
test_index = fnames[4000:4200]
'''
train_index = fnames[0:5]
test_index = fnames[10:15]



label_path = '../NewPNGlabeled/'
ori_path = '../JpegOriginalImg/'

train_input, train_label = read_images(label_path, ori_path, train_index, size)
test_input, test_label = read_images(label_path, ori_path, test_index, size)

In [ ]:
def gabor_filter(x,y,params):
    f,gamma,sigma,theta,psi = params
    xt = x*math.cos(theta) + y*math.sin(theta)
    yt = -x*math.sin(theta) + y*math.cos(theta)
    z1 = -(xt**2 + (gamma*yt)**2)/(2*sigma**2)
    z2 = 2*math.pi*f*xt+psi
    value = f**2/(math.pi*gamma)*math.exp(z1)*math.cos(z2)
    return value

def gabor_filter_dev(x,y,params):
    f,gamma,sigma,theta,psi = params
    z2 = psi
    value = f**2/(math.pi*gamma)*math.cos(z2)
    return value


def g_psi(x,y,params):
    f,gamma,sigma,theta,psi = params
    value = gabor_filter_dev(x,y,params)  
    return value

def gabor_filter_update_dev(size, params, f):
    bond = math.floor(size/2)
    x_range = np.linspace(-bond, bond, size)
    y_range = np.linspace(-bond, bond, size)

    [x_range,y_range] = list(map(lambda x:x.reshape(1,-1),np.meshgrid(x_range,y_range)))
    gfilter = []
    updates = []
    for (x,y) in zip(np.ndarray.tolist(x_range)[0], np.ndarray.tolist(y_range)[0]):        
        update = f(x,y,params)
        updates.append(update.real)
    updates = np.array(updates, dtype=np.float32).reshape(size,-1)
    return updates


def g_updates_dev(loss, params, gs):
    # Calculate gradients: 0.05
    gs_gradients = []
    gradients = nn.updates.get_or_compute_grads(loss, params)
    
    for w_index in range (0, int(len(params)/2)):
        # First Loop
        print(w_index)
        
        ws = params[w_index*2]
        g_params = gs[w_index].get_value()
        ws_gradients = gradients[w_index*2]

        [num_filters, num_channels, filter_size, filter_size] = ws.get_value().shape
        
        position = math.floor(filter_size/2)
        ws_grad = gradients[0][:,:,position:position+1,position]
        #g_gradient = g_psi(0,0,g)

        # Second and third Loop
        additions = []
        for i in range (0, num_filters):
            for j in range (0, num_channels):
                g = g_params[i,j,:]

                additions.append(g_psi(0,0,g))
                
                #addition = gabor_filter_update_dev(filter_size, g, g_psi)	# need adjustment
                
                
                #a = (w_gradient*addition).sum()
                #psi_gradient = a/9
                #g_gradient = np.array([0,0,0,0,psi_gradient.eval()]).reshape(1,-1)		# need adjustment				
                #g_gradients = np.concatenate((g_gradients,g_gradient),axis=1)
                
                #a = sum(sum(w_gradient*addition))
                #a = (w_gradient*addition).sum()
                #psi_gradient = a/9
                #g_gradient = np.array([update*0,update*0,update*0,update*0,update]).reshape(1,-1)
                #g_gradient = np.array([0,0,0,0,psi_gradient]).reshape(1,-1)
                #g_gradients = np.concatenate((g_gradients,g_gradient),axis=1)

        additions =  np.array(additions, dtype=np.float32).reshape(num_filters,num_channels,-1)
        g_gradients = theano.tensor.concatenate([ws_grad*0, ws_grad*0, ws_grad*0, ws_grad*0, ws_grad*0], axis=2)*additions
        print(additions.dtype)
        gs_gradients.append(g_gradients)
        
    gs_updates = nn.updates.adam_dev(gs_gradients, gs)
    return gs_updates

In [ ]:
start = timeit.default_timer()
gs_updates = g_updates_dev(loss, params, gs)
stop = timeit.default_timer()
print(stop - start)

In [ ]:
from collections import OrderedDict
updates = OrderedDict()
updates_old = nn.updates.adam(loss, params, learning_rate=lr)
for i in range (0,len(gs)):
    print(i)
    gs_new = gs_updates[gs[0]]
    ws = params[i*2]
    [num_filters, num_channels, filter_size, filter_size] = ws.get_value().shape
    W = gabor_weight_update([num_filters, num_channels, filter_size, filter_size], gs_new)
    #updates[ws] = theano.shared(W)
    updates[ws] = W
    updates[params[i*2+1]] = updates_old[params[i*2+1]]


In [ ]:
updates[params[0]].dtype

In [ ]:
start = timeit.default_timer()
w_index = 0
ws = params[w_index*2]
g_params = gs[w_index].get_value()
ws_gradients = gradients[w_index*2]

[num_filters, num_channels, filter_size, filter_size] = ws.get_value().shape

g_gradients = np.array([], dtype=np.float32).reshape(1,-1)

i = 0
j = 0
g = g_params[i,j,:]
w = ws[i,j,:,:]

w_gradient = ws_gradients[i,j,:]

addition = gabor_filter_update_dev(filter_size, g, g_psi)	# need adjustment
stop = timeit.default_timer()
print stop - start 

In [ ]:
gradients = nn.updates.get_or_compute_grads(loss, params)
ws_gradients = gradients[0]
ws_gradients[0,0,1,1]
theano.shared(ws_gradients)

In [ ]:
def gabor_filter_dev2(x,y,params):
    f,gamma,sigma,theta,psi = params
    xt = x*theano.tensor.cos(theta) + y*theano.tensor.sin(theta)
    yt = -x*theano.tensor.sin(theta) + y*theano.tensor.cos(theta)
    z1 = -(xt**2 + (gamma*yt)**2)/(2*sigma**2)
    z2 = 2*math.pi*f*xt+psi
    #z2 = 1j*2*math.pi*f*xt+psi
    value = f**2/(math.pi*gamma)*theano.tensor.exp(z1)*theano.tensor.cos(z2)
    return value

def gabor_filter_1(x,y,params):
    f,gamma,sigma,theta,psi = params
    xt = x*math.cos(theta) + y*math.sin(theta)
    yt = -x*math.sin(theta) + y*math.cos(theta)
    z1 = -(xt**2 + (gamma*yt)**2)/(2*sigma**2)
    z2 = 1j*2*math.pi*f*xt+psi*1j
    value = f**2/(math.pi*gamma)*math.exp(z1)*cmath.exp(z2)
    return value

def gabor_filter_2(x,y,params):
    f,gamma,sigma,theta,psi = params
    xt = x*math.cos(theta) + y*math.sin(theta)
    yt = -x*math.sin(theta) + y*math.cos(theta)
    z1 = -(xt**2 + (gamma*yt)**2)/(2*sigma**2)
    z2 = 2*math.pi*f*xt+psi
    value = f**2/(math.pi*gamma)*math.exp(z1)*math.cos(z2)
    return value

In [ ]:
w_gradients = np.ones([4,4,5])
trys = theano.shared(w_gradients)
#trys.eval()
g_try = gs_updates[gs[0]]
#a = gs[0][0,0,1]
a = g_try[0,0][1]
params = [a,a,a,a,a]
#filter_ps = [gabor_filter(0,0,params),gabor_filter(0,0,params),gabor_filter(0,0,params),gabor_filter(0,0,params)]
#np.array(filter_ps)
x = gabor_filter_dev2(0,0,params)
lis = []
for i in range(0,9):
    lis.append(x)
y = theano.tensor.stack(lis)
y = y.reshape((3,3))
y.dtype

In [ ]:
g_try = gs_updates[gs[0]]
updates
g_try.dtype

In [9]:
gs_updates.keys()

odict_keys([<TensorType(float32, (False, True, False))>, <TensorType(float32, (False, True, False))>, g, <TensorType(float32, (True, False, False))>, <TensorType(float32, (True, False, False))>, g, <TensorType(float32, scalar)>])

In [ ]:
updates_old = nn.updates.adam(loss, params, learning_rate=lr)
updates_old[params[0]].dtype

In [ ]:
gvalue = gs[0].get_value()
g = gvalue[0,0,:]
g_psi(0,0,g)
